In [29]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer 
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler

In [30]:
df_original = pd.read_csv('/content/drive/MyDrive/IA y Ciencia de Datos/Semestre 3/Aprendizaje Automatico 1/2da_Instancia_Evaluativa-AA_1/Data/Copia de Titanic-Dataset.csv')
df_original

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [31]:
df_original.nunique() # Utilizo esto para ver todos los datos que son unicos asi me hago una idea de lo que puedo usar y lo que no

PassengerId    891
Survived         2
Pclass           3
Name           891
Sex              2
Age             88
SibSp            7
Parch            7
Ticket         681
Fare           248
Cabin          147
Embarked         3
dtype: int64

##Tratamiento de los datos


In [32]:
df_original.info() # Viendo esto me asusta la 10 cabin y la edad es preocupante que le falten tantos datos

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


Viendo esto puedo decir que 'passangerID' esta al pedo ya que le DF ya tiene su propio index, 'Cabin' es inutil teniendo tantos datos faltantes, tratar de rellenarlo seria especular demasiado, Name es unica para cada persona y no me dice nada si sobrevive o no y ticket demasiado compleja para usarla las voy a sacar

### Columnas Cabin, PassengerId, Nombre y Ticket

In [33]:
df_original = df_original.drop(['Cabin','PassengerId','Name','Ticket'],axis=1) # Elimino las culumnas con las que no voy a trabajar

### Relleno de Datos vacios
Embarked tiene 2 Datos Vacios y la edad tiene varios

**Embarked**

In [34]:
#Embarked Primero 
df_original['Embarked'].unique() # busco averiguar que tipo de dato es, y como esta conformado

array(['S', 'C', 'Q', nan], dtype=object)

In [35]:
df_original['Embarked'].groupby(df_original['Embarked']).count()# una ves vi eso quiero saber cuanto hay de cada tipo de dato

Embarked
C    168
Q     77
S    644
Name: Embarked, dtype: int64

In [36]:
#Viendo el resultado anterior decido antes que rellenarlo con un imputer es mas facil hacerlo a mano
df_original['Embarked'] = df_original['Embarked'].fillna('S')#como la mayoria de los datos son 'S' me la juego a que esos dos tambien son 'S'
df_original.info() # Verifico que ahora son 891 datos no nulos. 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  891 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB


**Edad**

In [37]:
Clases = df_original.groupby('Pclass')['Age'].transform('mean') # Creo un promedio de las edades de los participantes segun su clase
Clases

0      25.140620
1      38.233441
2      25.140620
3      38.233441
4      25.140620
         ...    
886    29.877630
887    38.233441
888    25.140620
889    38.233441
890    25.140620
Name: Age, Length: 891, dtype: float64

In [38]:
df_original['Age'] = df_original['Age'].fillna(Clases) # y se lo asigno a la columna Age con un fillna

In [39]:
df_original['Age'].unique()#Verifico que no cometi un error y no cambie todos los valores por la media

array([22.        , 38.        , 26.        , 35.        , 25.14061972,
       54.        ,  2.        , 27.        , 14.        ,  4.        ,
       58.        , 20.        , 39.        , 55.        , 29.87763006,
       31.        , 34.        , 15.        , 28.        ,  8.        ,
       19.        , 40.        , 38.23344086, 66.        , 42.        ,
       21.        , 18.        ,  3.        ,  7.        , 49.        ,
       29.        , 65.        , 28.5       ,  5.        , 11.        ,
       45.        , 17.        , 32.        , 16.        , 25.        ,
        0.83      , 30.        , 33.        , 23.        , 24.        ,
       46.        , 59.        , 71.        , 37.        , 47.        ,
       14.5       , 70.5       , 32.5       , 12.        ,  9.        ,
       36.5       , 51.        , 55.5       , 40.5       , 44.        ,
        1.        , 61.        , 56.        , 50.        , 36.        ,
       45.5       , 20.5       , 62.        , 41.        , 52.  

### Graficas de Comparacion de las variables y la taza de supervivencia
Busco encontrar alguna similitud entre los datos y la taza de supervicencia

In [40]:
df_original.info()# lo uso para guiarme y usar todas las filas

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       891 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  891 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB


In [41]:
import plotly.express as px
fig = px.histogram(df_original, x='Pclass', y='Survived',color='Sex',barmode='group',title='Relacion entre Clase y taza de supervivencia')
fig.show()
#se puede ver como la taza de supervivencia es mucho mayor en muyeres de todas las clases, aunque me sorprende que los hombres de la clase 3 hayan sobrevivido mas.

In [42]:
fig = px.histogram(df_original, x='Age', y='Survived',color='Sex',barmode='group',title='Relacion entre edad, sexo y taza de supervivencia')
fig.show()
#Es de esperar tambien que la mayoria de personas que viajaron son de esa edad

In [43]:
df_original.groupby('Sex')['Age'].mean()
#Como dije arriba, si, la mayoria estaba entre 20 y 30

Sex
female    27.852746
male      30.076584
Name: Age, dtype: float64

In [44]:
fig = px.histogram(df_original, x='SibSp', y='Survived',color='Sex',barmode='group',title='Relacion entre cantidad de Familiares y taza de supervivencia')
fig.show()
#Aca me sorprende ver que mientras mas familiares tenias, era mas probable morir.

In [45]:
fig = px.histogram(df_original, x='Parch', y='Survived',color='Sex',barmode='group',title='Relacion entre cantidad de Familiares y taza de supervivencia')
fig.show()

Viendo los dos ultimos histogramas puedo ver que hay una alta relacion entre ellos, claramente son lo mismo y podria sumarlos, pero tambien puedo ver que hay mas cantidad de supervivientes de la gente que no tenia familiares.

### Modificacion de columnas

In [46]:
#Buscando en internet y sus explicaciones encuentro que Parch y SibSp son familiares 
df_original['Familiares'] = df_original['Parch'] + df_original['SibSp']
df_original = df_original.drop(['SibSp','Parch'],axis=1)
df_original

,Survived,Pclass,Sex,Age,Fare,Embarked,Familiares
0,0,3,male,22.00000,7.2500,S,1
1,1,1,female,38.00000,71.2833,C,1
2,1,3,female,26.00000,7.9250,S,0
3,1,1,female,35.00000,53.1000,S,1
4,0,3,male,35.00000,8.0500,S,0
...,...,...,...,...,...,...,...
886,0,2,male,27.00000,13.0000,S,0
887,1,1,female,19.00000,30.0000,S,0
888,0,3,female,25.14062,23.4500,S,3
889,1,1,male,26.00000,30.0000,C,0


In [47]:
df_original['Viaja_Solo'] = df_original['Familiares'] == 0 #Creo una columna en la que se puede ver que viaja solo o no para que el modelo lo aprecie
df_original

,Survived,Pclass,Sex,Age,Fare,Embarked,Familiares,Viaja_Solo
0,0,3,male,22.00000,7.2500,S,1,False
1,1,1,female,38.00000,71.2833,C,1,False
2,1,3,female,26.00000,7.9250,S,0,True
3,1,1,female,35.00000,53.1000,S,1,False
4,0,3,male,35.00000,8.0500,S,0,True
...,...,...,...,...,...,...,...,...
886,0,2,male,27.00000,13.0000,S,0,True
887,1,1,female,19.00000,30.0000,S,0,True
888,0,3,female,25.14062,23.4500,S,3,False
889,1,1,male,26.00000,30.0000,C,0,True


**Creando el Train y el Test**

In [48]:
y = df_original['Survived']
x = df_original.drop(['Survived'],axis=1) #Separo con pandas en X e Y

In [49]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=123) # lo divido por primera vez en train y test

In [50]:
Valores_Embarked=['S','C','Q']
Tonehot = ("onehot",OneHotEncoder(sparse=False), ['Viaja_Solo','Sex'])
Tordinal =("ordinal",OrdinalEncoder(categories=[Valores_Embarked] ), ['Embarked'])
Tnorm = ("normalizar",MinMaxScaler(feature_range=(0, 1)),['Age','Fare','Pclass']) # Creo los modelos para modificar las columnas

In [51]:
transformador_columnas = ColumnTransformer(transformers=[Tordinal,Tonehot,Tnorm],remainder='passthrough') # Creo el mod de columnas con los modelos anteriores

In [52]:
transformador_columnas.fit(X_train) # lo entreno con el train

ColumnTransformer(remainder='passthrough',
                  transformers=[('ordinal',
                                 OrdinalEncoder(categories=[['S', 'C', 'Q']]),
                                 ['Embarked']),
                                ('onehot', OneHotEncoder(sparse=False),
                                 ['Viaja_Solo', 'Sex']),
                                ('normalizar', MinMaxScaler(),
                                 ['Age', 'Fare', 'Pclass'])])

In [53]:
X_train_T = pd.DataFrame(transformador_columnas.transform(X_train))
X_test_T = pd.DataFrame(transformador_columnas.transform(X_test)) # modifico los datos del Test y train en X

In [54]:
X_train_T #Veo que Este todo bien

,0,1,2,3,4,5,6,7,8
0,1.0,1.0,0.0,1.0,0.0,0.195778,0.113168,0.0,1.0
1,2.0,0.0,1.0,0.0,1.0,0.384267,0.015127,1.0,0.0
2,1.0,0.0,1.0,0.0,1.0,0.566474,0.014102,1.0,0.0
3,2.0,0.0,1.0,0.0,1.0,0.258608,0.015094,1.0,0.0
4,1.0,1.0,0.0,0.0,1.0,0.283740,0.123667,0.0,1.0
...,...,...,...,...,...,...,...,...,...
707,0.0,1.0,0.0,1.0,0.0,0.421965,0.044893,0.5,1.0
708,2.0,0.0,1.0,1.0,0.0,0.371701,0.024106,0.5,0.0
709,0.0,0.0,1.0,0.0,1.0,0.396833,0.015469,1.0,0.0
710,0.0,0.0,1.0,0.0,1.0,0.371701,0.014151,1.0,0.0


Creo los Validation 

In [55]:
X_Val_train, X_Val_test, y_Val_train, y_Val_test = train_test_split(X_train_T, y_train, test_size=0.2, random_state=123) #Creo el Valtest y el ValTrain

## Modelos de ML

### Random Forest


> Este modelo es el que voy a usar ya que es el mas rapido (aunque no por mucho) de todos y me da el mismo Accuracy que los otros



In [56]:
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score

In [57]:
from sklearn.ensemble import RandomForestClassifier

In [64]:
n_estimators = np.arange(10,50,10)
max_depth = np.arange(5,15)
criterion = ["gini", "entropy"]
min_samples_split = np.arange(2,5)
df_RandomF = pd.DataFrame(columns=['Accuracy','min_samples_split','Criterion','n_Estimators','max_depth'])
for i in n_estimators:
    for j in criterion:
      for k in min_samples_split:
          for a in max_depth:
              rf = RandomForestClassifier(min_samples_split=k,criterion = j, n_estimators= i,max_depth=a, random_state=123)
              rf.fit(X_Val_train, y_Val_train)
              y_pred_rf = rf.predict(X_Val_test)
              acc_rf = accuracy_score(y_Val_test, y_pred_rf)
              df_RandomF = df_RandomF.append({'Accuracy': acc_rf,'max_depth': a,'Criterion': j,'n_Estimators': i,'min_samples_split':k}, ignore_index=True)

In [65]:
df_RandomF.sort_values(by=['Accuracy'],ascending=False).head(30)

,Accuracy,min_samples_split,Criterion,n_Estimators,max_depth
41,0.888112,3,entropy,10,6
194,0.881119,3,gini,40,9
74,0.874126,3,gini,20,9
181,0.874126,2,gini,40,6
215,0.874126,2,entropy,40,10
200,0.874126,4,gini,40,5
190,0.874126,3,gini,40,5
210,0.874126,2,entropy,40,5
151,0.874126,2,entropy,30,6
4,0.874126,2,gini,10,9


### Gradient

In [60]:
from sklearn.ensemble import GradientBoostingClassifier

In [61]:
#loss = ['exponential','log_loss'] #default log_loss
criterion = ['friedman_mse', 'squared_error']
min_samples_split = np.arange(2,12,1)
# learning_rate =np.arange(0.1,10.1,1) # default=0.1 desde/hasta:(0.0, inf).
n_estimators =np.arange(18,25,1) #default=100 desde/hasta: [1, inf).
max_depth = np.arange(5,14,1) #default=3

In [62]:
df_gb = pd.DataFrame(columns=['Accuracy','Max_depth','Criterion','Min_Samples_Split','Min_Sample_Leaf','Learning_Rate','n_Estimators'])
for x in max_depth:
    for b in criterion:
      for c in min_samples_split:
          for e in n_estimators:
                    gb = GradientBoostingClassifier(max_depth=x,
                                                    criterion=b,
                                                    min_samples_split=c,
                                                    learning_rate=0.1,
                                                    n_estimators=e,
                                                    min_samples_leaf=8,
                                                    random_state = 123)
                    gb.fit(X_Val_train, y_Val_train)
                    y_pred_gb = gb.predict(X_Val_test)
                    acc_gb = accuracy_score(y_Val_test, y_pred_gb)
                    df_gb = df_gb.append({'Accuracy': acc_gb,'Max_depth': x,'Criterion':b,'Min_Samples_Split':c,'Min_Sample_Leaf':8,'Learning_Rate':0.1,'n_Estimators':e}, ignore_index=True)

KeyboardInterrupt: ignored

In [ ]:
df_gb.sort_values(by=['Accuracy'],ascending=False)

### XGBoost

In [ ]:
from xgboost import XGBClassifier

In [ ]:
n_estimators = np.arange(1,10,1)
max_depth = np.arange(3,10,1)
learning_rate = np.arange(0.1,1,0.1)
subsample = np.arange(0.9,1,0.01)
colsample_bytree = np.arange(0.9,1,0.01)

In [ ]:
df_xgboost = pd.DataFrame(columns=['Accuracy','n_Estimators','max_depth','Learning_Rate','Subsample','Colsample_bytree'])
for a in max_depth:
    for b in n_estimators:
        for c in learning_rate:
            for d in subsample:
                for e in colsample_bytree:
                    xgbo = XGBClassifier(max_depth = a,
                                         n_estimators = b,
                                         learning_rate = c,
                                         subsample = d,
                                         colsample_bytree = e,
                                         objective = 'binary:logistic', 
                                         eval_metric = 'logloss')
                    xgbo.fit(X_Val_train, y_Val_train)
                    y_pred_xgbo = xgbo.predict(X_Val_test)
                    acc_xgbo = accuracy_score(y_Val_test, y_pred_xgbo)
            df_xgboost = df_xgboost.append({'Accuracy':acc_xgbo,'n_Estimators':b,'max_depth':a,'Learning_Rate':c, 'Subsample':d,'Colsample_bytree':e}, ignore_index=True)

In [ ]:
df_xgboost.sort_values(by=['Accuracy'],ascending=False)

## Modelo Final Con RandomForest

In [69]:
# Una vez encontrado los hiperparametros que queria simplemente los puse y vi el resultado
rf = RandomForestClassifier(min_samples_split= 3,criterion ='entropy', n_estimators= 10,max_depth=6, random_state=123)
rf.fit(X_train_T, y_train)
y_pred_rf = rf.predict(X_test_T)
acc_rf = accuracy_score(y_test, y_pred_rf)
print(acc_rf)

0.8324022346368715
